In [ ]:
# dev/inference.ipynb
#import mlflow
#import pandas as pd

#print("DEV INFERENCE NOTEBOOK")

#model_name = "my_classification_model_dev"  # dev-specific model name
#model_uri = f"models:/{model_name}/Staging" # dev might store in 'Staging' or 'None'

#model = mlflow.pyfunc.load_model(model_uri)

#df = pd.DataFrame({
#    "feature1": [5.1, 4.9],
#    "feature2": [3.5, 3.0],
#    "feature3": [1.4, 1.4],
#    "feature4": [0.2, 0.2]
#})

#predictions = model.predict(df)
#print("Dev inference predictions:", predictions)


In [ ]:
%pip install mlflow>=2.0.0 scikit-learn>=1.0.0 pandas>=1.3.0

In [ ]:
dbutils.library.restartPython()

In [ ]:
import mlflow
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from datetime import datetime
import os

In [ ]:
mlflow.set_registry_uri("databricks-uc")

In [ ]:
#loading of parameters
catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")
model_name = dbutils.widgets.get("model_name")
version = dbutils.widgets.get("version")

In [ ]:
#"models:/prod_catalog.ml_models.iris_model@production"
model_to_load = f"models:/{catalog_name}.{schema_name}.{model_name}@{version}"

In [ ]:
model = mlflow.pyfunc.load_model(model_to_load)

In [ ]:
iris = load_iris()
inference_data = pd.DataFrame(iris.data[:150], columns=iris.feature_names)


In [ ]:
expected_schema = {
    'sepal length (cm)': 'float64',
    'sepal width (cm)': 'float64', 
    'petal length (cm)': 'float64',
    'petal width (cm)': 'float64'
}

In [ ]:
if inference_data.dtypes.astype(str).to_dict() != expected_schema:
    dbutils.notebook.exit("Schema validation failed")

In [ ]:
predictions = model.predict(inference_data)
inference_data['prediction'] = predictions

In [ ]:
date_str = datetime.today().strftime("%Y-%m-%d")
output_dir = f"/Volumes/prod_catalog/ml_models/inference_results/{date_str}"
output_path = f"{output_dir}/predictions.csv"

# Create the directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

inference_data.to_csv(output_path, index=False)